In [3]:
import openai
import os

from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
import warnings

warnings.filterwarnings('ignore')

In [5]:
llm_model = "gpt-3.5-turbo"

In [6]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
df = pd.read_csv("Data.csv")

In [8]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [9]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [10]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [11]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [12]:
chain = LLMChain(llm=llm, prompt=prompt)

In [13]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Royal Comfort Bedding"'

In [14]:
from langchain.chains import SimpleSequentialChain

In [15]:
llm = ChatOpenAI(temperature=0.9, model_name=llm_model)

first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}"
)

# chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(
    chains= [chain_one,chain_two],
    verbose= True
)

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
"Queen Comfort Co."
Queen Comfort Co. offers luxurious and stylish bedding and home textiles designed for the queen in every woman, combining quality and elegance.

> Finished chain.


'Queen Comfort Co. offers luxurious and stylish bedding and home textiles designed for the queen in every woman, combining quality and elegance.'

In [19]:
from langchain.chains import SequentialChain

In [24]:
llm = ChatOpenAI(temperature=0.9, model_name = llm_model)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

In [25]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key= "summary")

In [22]:
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review: \n\n{Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key= "language")

In [29]:
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=llm, 
    prompt=fourth_prompt,
    output_key="followup_message"
)

In [30]:
overall_chain = SequentialChain(
    chains = [chain_one, chain_two, chain_three, chain_four],
    input_variables= ["Review"],
    output_variables= ["English_Review","summary","followup_message"],
    verbose= True
)

In [31]:
review= df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'English_Review': "I find the taste mediocre. The foam doesn't hold, it's weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': 'The reviewer finds the taste of the product mediocre, with poorly holding foam, leading them to suspect that the product may be an old batch or counterfeit.',
 'followup_message': "Je suis désolé d'entendre que vous avez trouvé le goût du produit médiocre et que la mousse ne tient pas bien. Il est possible que vous ayez reçu un lot ancien ou contrefait. Il pourrait être utile de contacter le fabricant pour clarifier la situation et peut-être obtenir un remplacement ou un remboursement. Merci de partager votre expérience avec nous."}

In [33]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [34]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [45]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import ChatPromptTemplate, PromptTemplate

In [36]:
llm = ChatOpenAI(temperature=0.0, model_name= llm_model)

In [39]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)

    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions
computer science: Good for answering computer science questions


In [40]:
print(type(destinations_str), type(destinations))

<class 'str'> <class 'list'>


In [41]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [42]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [46]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

router_prompt = PromptTemplate(
    template= router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [47]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains= destination_chains,
    default_chain= default_chain,
    verbose= True 
)

In [48]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [49]:
chain.run("Give some example of black body radiation")



> Entering new MultiPromptChain chain...
physics: {'input': 'Give some examples of black body radiation'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which absorbs all incident radiation and emits radiation at all wavelengths. Some examples of black body radiation include:\n\n1. The radiation emitted by a heated filament in an incandescent light bulb.\n2. The radiation emitted by the Sun, which closely approximates a black body radiator.\n3. The radiation emitted by a heated metal object, such as a stove burner or a piece of metal in a blacksmith's forge.\n4. The cosmic microwave background radiation, which is the remnant radiation from the Big Bang and is often modeled as black body radiation."

In [50]:
chain.run("What is great depression?")



> Entering new MultiPromptChain chain...
History: {'input': 'What is great depression?'}
> Finished chain.


'The Great Depression was a severe worldwide economic downturn that lasted from 1929 to the late 1930s. It was the longest, deepest, and most widespread depression of the 20th century. The Great Depression was triggered by the stock market crash of 1929, which led to widespread unemployment, poverty, and a sharp decline in industrial production. It had devastating effects on the global economy, leading to widespread hardship and suffering for millions of people. The Great Depression ultimately led to significant changes in economic policy and regulation to prevent such a catastrophic event from happening again.'

In [51]:
chain.run("What is the sum of 2 + 3?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is the sum of 2 + 3?'}
> Finished chain.


'The sum of 2 + 3 is 5.'

In [52]:
chain.run("What is the symbolic representation of salfuric acid")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the symbolic representation of sulfuric acid'}
> Finished chain.


'The symbolic representation of sulfuric acid is H2SO4.'

In [53]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of an organism. DNA contains the instructions for building and maintaining an organism, including the proteins that are essential for cell structure and function. This genetic information is passed down from parent to offspring and is essential for the growth, development, and functioning of all cells in the body. Having DNA in every cell ensures that each cell has the necessary information to carry out its specific functions and contribute to the overall functioning of the organism.'

In [55]:
chain.run("What is the most popular ai agent framworks in 2025?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the most popular AI agent frameworks in 2025?'}
> Finished chain.


'As of 2025, some of the most popular AI agent frameworks include TensorFlow, PyTorch, OpenAI, Microsoft Cognitive Toolkit, and IBM Watson. These frameworks are widely used in various industries for developing AI applications, machine learning models, natural language processing, computer vision, and more. Each framework has its own strengths and weaknesses, so the choice of framework often depends on the specific requirements of the project. Additionally, new frameworks may emerge in the future as the field of AI continues to evolve.'